### Import statements

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import palettable as pltt
import seaborn as sns
from sklearn import preprocessing
import random

In [3]:
# Install a conda package: pandas-profiling in the current Jupyter kernel
import sys
!conda install --yes --prefix {sys.prefix} pandas-profiling

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\timon\anaconda3\envs\gds

  added / updated specs:
    - pandas-profiling


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.07.19 |       haa95532_0         123 KB
    certifi-2022.9.24          |   py39haa95532_0         154 KB
    conda-22.9.0               |   py39haa95532_0         888 KB
    confuse-1.5.0              |     pyhd3eb1b0_0          23 KB
    htmlmin-0.1.12             |     pyhd3eb1b0_1          24 KB
    imagehash-4.2.1            |     pyhd3eb1b0_0         285 KB
    missingno-0.4.2            |     pyhd3eb1b0_1          15 KB
    openssl-1.1.1q             |       h2bbff1b_0         4.8 MB
    pandas-profiling-2.9.0     |     pyhd3eb1b0_0         168 KB
    phik-0.11.2                |     pyhd3eb1b0_1         538 KB
    pywavelets-1.3.0           |   p

In [7]:
import pandas_profiling as pp

ImportError: cannot import name 'ABCIndexClass' from 'pandas.core.dtypes.generic' (C:\Users\timon\anaconda3\envs\gds\lib\site-packages\pandas\core\dtypes\generic.py)

In [5]:
pip install dtale

     ---------------------------------------- 12.8/12.8 MB 6.2 MB/s eta 0:00:00
     -------------------------------------- 642.7/642.7 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 65.9/65.9 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 15.2/15.2 MB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 9.8/9.8 MB 6.8 MB/s eta 0:00:00
     -------------------------------------- 101.5/101.5 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 62.3/62.3 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 45.9/45.9 kB 2.2 MB/s eta 0:00:00
     -------------------------------------- 216.1/216.1 kB 4.4 MB/s eta 0:00:00
     -------------------------------------- 232.7/232.7 kB 4.8 MB/s eta 0:00:0

# Load, clean and rename data

## 1. Emergency calls data
For explanation of features, <a href="https://cusp.tbm.tudelft.nl/courses/epa1316/project/project-02.pdf"> click here </a>

In [6]:
# Load 112_calls_data.csv
em_calls = pd.read_csv("data/112_calls_data.csv", sep = ",")

In [ ]:
print('{} different cities'.format(len(em_calls['wplNam'].unique())))
print('{} different municipalities'.format(len(em_calls['gemName'].unique())))
em_calls.shape

In [ ]:
print('For The Hague:')
print('\n{}\n'.format(th.shape))
th = em_calls[em_calls['wplNam']=='Den Haag'] # the hague
print('priority\tcount')
for prio in th['pmePrioLevel'].unique():
    count = len(th[th['pmePrioLevel']==prio])
    print('{}\t\t{}'.format(prio, count))

th.head(2)

In [8]:
import dtale

In [9]:
dtale.show(em_calls)

C:\Users\timon\anaconda3\envs\gds\lib\site-packages\dtale\views.py:725: FutureWarning:

['pmeStrippedMessage'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.



## 2. The Hague demographic data
Find dashboard with these variables <a href="https://denhaag.incijfers.nl/Jive?workspace_guid=83bb13aa-9f6f-49ab-ba68-f8fb2ffa2e34"> here </a>. They all refer to households, not persons, except 65 and older.

In [ ]:
# load CBS data per neighborhood
data = pd.read_csv("data/th_neighborhoods_demographics.csv", sep = ";")

# drop 'Onbekend'
i = data.loc[data['Buurten'] == 'Onbekend'].index
data.drop(index=i, inplace=True)

# translate column labels
dictionary = {'Buurten': 'Neighborhood', 
        '65 jaar en ouder': '65_older_persons',
        "% 65 jaar en ouder":'65_older_%',
        'Verhouding b.v.o. medische voorzieningen/ totale gebiedsoppervlakte': 'medical_fac_rel_area',
        'Verhouding b.v.o. sportvoorzieningen/ totale gebiedsoppervlakte': 'sports_fac_rel_area',
        'Verhouding b.v.o. recreatieve voorzieningen/ totale gebiedsoppervlakte': 'leisure_fac_rel_area',
        'Gem. besteedbaar gestandaardiseerd part. huishoudens inkomen': 'disposable_income_std',
        'Aantal huishoudens': 'households_count',
        '% Met migratieachtergrond': 'migration_background_%',
        'Gemiddelde achterstandsscore': 'disadvantage_score',
        'Totaal doelgroephuishoudens met inkomen tot 130%':'130%_poverty_line_count'}
data.rename(columns=dictionary, inplace=True)
 
# set Neighborhood as index
data = data.set_index('Neighborhood', drop=True)

# make strings numerical
data.replace(',', '.', regex=True, inplace=True) # comma -> point
data.replace('x', 'NaN', regex=True, inplace=True) # 'x' -> NaN
data.replace('-', 'NaN', regex=False, inplace=True) # 'x' -> NaN

for i in range(6):
    data.iloc[:,i] = data.iloc[:,i].astype(float)

# check outcome
data.info()
data.head(5)

In [ ]:
test = gpd.GeoDataFrame()
test.head()